In [0]:
# In a new cell
%pip install -U threadpoolctl torch shap
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text('features_table', "titanic_features")
features_table = dbutils.widgets.get('features_table')
pdf = spark.sql(f"SELECT * FROM dp_ml_raw.features.{features_table}").toPandas()
cols = pdf.columns
print(cols, len(cols))

In [0]:
# --- Load features (includes target) ---
# TABLE = "dp_ml_raw.features.titanic_features"
# pdf = spark.table(TABLE).toPandas()

y = pdf['num__Survived'].astype(int)                           # ensure int/binary target

# Keep only numeric features; drop target and any obvious keys
drop_cols = ["rowId", "num__Survived"]
X = pdf.drop(columns=drop_cols)
# X = X.select_dtypes(include=["number"]).copy()        # features must be numeric for sklearn

print(f"X shape: {X.shape}, y shape: {y.shape}")

# --- 3-fold CV with train vs validation log-loss (cross-entropy) ---
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import StandardScaler
import numpy as np

folds = 10
skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
train_losses, val_losses, train_accs, val_accs = [], [], [], []

fold = 0
for train_idx, val_idx in skf.split(X, y):
    fold += 1
    X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]

    sc = StandardScaler(); X_tr = sc.fit_transform(X_tr).astype("float32"); X_va = sc.transform(X_va).astype("float32")

    model = LogisticRegression(solver="lbfgs", max_iter=1000)
    model.fit(X_tr, y_tr)

    # Predict probabilities for log-loss
    p_tr = model.predict_proba(X_tr)[:, 1]
    p_va = model.predict_proba(X_va)[:, 1]

    tr_loss = log_loss(y_tr, p_tr)
    va_loss = log_loss(y_va, p_va)
    train_losses.append(tr_loss)
    val_losses.append(va_loss)

    yhat_tr = model.predict(X_tr)
    yhat_va = model.predict(X_va)
    tr_acc = accuracy_score(y_tr, yhat_tr)
    va_acc = accuracy_score(y_va, yhat_va)
    train_accs.append(tr_acc)
    val_accs.append(va_acc)

    print(f"Fold {fold}: "
          f"train log-loss={tr_loss:.4f}, val log-loss={va_loss:.4f} | "
          f"train acc={tr_acc:.4f}, val acc={va_acc:.4f}")

print(f"\n=== {folds}-fold summary ===")
print(f"Train log-loss: mean={np.mean(train_losses):.4f} ± {np.std(train_losses):.4f}")
print(f"Valid log-loss: mean={np.mean(val_losses):.4f} ± {np.std(val_losses):.4f}")
print(f"Train accuracy: mean={np.mean(train_accs):.4f} ± {np.std(train_accs):.4f}")
print(f"Valid accuracy: mean={np.mean(val_accs):.4f} ± {np.std(val_accs):.4f}")


# Fit final model on all data (optional sanity checks)
final_model = LogisticRegression(solver="lbfgs", max_iter=1000).fit(X, y)


In [0]:
import numpy as np, torch, torch.nn as nn
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# X: numeric DataFrame, y: {0,1} Series
Xn, yn = X.values.astype("float32"), y.values.astype("float32").reshape(-1,1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS, FOLDS = 500, 5

in_dim = Xn.shape[1]
def make_model(h=64):
    return nn.Sequential(nn.Linear(in_dim,h), nn.ReLU(), nn.Linear(h,1), nn.Sigmoid()).to(device)

crit = nn.BCELoss()
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

tr_all, va_all = [], []
tr_loss_final, va_loss_final = [], []
tr_acc_final,  va_acc_final  = [], []

for i, (tr_idx, va_idx) in enumerate(skf.split(Xn, yn.ravel()), 1):
    Xtr, Xva = Xn[tr_idx], Xn[va_idx]; ytr, yva = yn[tr_idx], yn[va_idx]
    sc = StandardScaler(); Xtr = sc.fit_transform(Xtr).astype("float32"); Xva = sc.transform(Xva).astype("float32")

    Xtr_t = torch.from_numpy(Xtr).to(device); ytr_t = torch.from_numpy(ytr).to(device)
    Xva_t = torch.from_numpy(Xva).to(device); yva_t = torch.from_numpy(yva).to(device)

    m = make_model(); opt = torch.optim.Adam(m.parameters(), lr=1e-3)
    tr_curve, va_curve = [], []

    for _ in range(EPOCHS):
        m.train(); opt.zero_grad()
        p = m(Xtr_t); loss = crit(p, ytr_t); loss.backward(); opt.step()
        tr_curve.append(loss.item())

        m.eval()
        with torch.no_grad():
            pv = m(Xva_t); va_curve.append(crit(pv, yva_t).item())

    # final epoch metrics
    with torch.no_grad():
        p  = m(Xtr_t); pv = m(Xva_t)
        tr_acc = ( (p  >= 0.5).float().eq(ytr_t).float().mean().item() )
        va_acc = ( (pv >= 0.5).float().eq(yva_t).float().mean().item() )

    tr_all.append(tr_curve); va_all.append(va_curve)
    tr_loss_final.append(tr_curve[-1]); va_loss_final.append(va_curve[-1])
    tr_acc_final.append(tr_acc);       va_acc_final.append(va_acc)

    print(f"Fold {i}: train loss={tr_curve[-1]:.4f}, val loss={va_curve[-1]:.4f}, "
          f"train acc={tr_acc:.4f}, val acc={va_acc:.4f}")

# ---- Averages across folds ----
print("\n=== Averages across folds ===")
print(f"Train loss: {np.mean(tr_loss_final):.4f} ± {np.std(tr_loss_final):.4f}")
print(f"Val   loss: {np.mean(va_loss_final):.4f} ± {np.std(va_loss_final):.4f}")
print(f"Train acc : {np.mean(tr_acc_final):.4f} ± {np.std(tr_acc_final):.4f}")
print(f"Val   acc : {np.mean(va_acc_final):.4f} ± {np.std(va_acc_final):.4f}")

# ---- Plot mean loss vs epoch ----
tr_all, va_all = np.array(tr_all), np.array(va_all)
epochs = np.arange(1, EPOCHS+1)
plt.figure()
plt.plot(epochs, tr_all.mean(0), label="Train loss")
plt.plot(epochs, va_all.mean(0), label="Val loss")
plt.fill_between(epochs, tr_all.mean(0)-tr_all.std(0), tr_all.mean(0)+tr_all.std(0), alpha=0.2)
plt.fill_between(epochs, va_all.mean(0)-va_all.std(0), va_all.mean(0)+va_all.std(0), alpha=0.2)
plt.xlabel("Epoch"); plt.ylabel("Binary cross-entropy")
plt.title(f"{FOLDS}-fold mean loss per epoch"); plt.legend(); plt.show()



In [0]:
# assumes: trained PyTorch model `m` (ends with nn.Sigmoid),
#          fitted scaler `sc`, and pandas DataFrame `X` (features)
import numpy as np, torch, shap, matplotlib.pyplot as plt, torch.nn as nn

m.eval()
device = next(m.parameters()).device

# copy model WITHOUT final Sigmoid for explaining logits (more stable)
logit_model = nn.Sequential(*list(m.children())[:-1]).to(device).eval()

# scale data (same scaler as training)
Xs = sc.transform(X.values.astype("float32"))
rng = np.random.default_rng(42)
bg_idx = rng.choice(len(Xs), size=min(200, len(Xs)), replace=False)
ex_idx = rng.choice(len(Xs), size=min(1000, len(Xs)), replace=False)

bg_t = torch.from_numpy(Xs[bg_idx]).to(device)
ex_t = torch.from_numpy(Xs[ex_idx]).to(device)

# --- SHAP DeepExplainer (disable additivity check at call time) ---
explainer = shap.DeepExplainer(logit_model, bg_t)
sv = explainer.shap_values(ex_t, check_additivity=False)  # <- key change

# Normalize to (N, F) numpy
if isinstance(sv, list): sv = sv[0]
if torch.is_tensor(sv):  sv = sv.detach().cpu().numpy()
sv = np.asarray(sv)
sv = np.squeeze(sv)                     # handles (N,F,1)/(N,1,F) → (N,F)
if sv.ndim == 1: sv = sv.reshape(-1, 1)

# Global importance = mean |SHAP| per feature
mean_abs = np.mean(np.abs(sv), axis=0)
feat_names = list(X.columns)
order = np.argsort(mean_abs)[::-1]

plt.figure(figsize=(10, max(3, 0.4*len(feat_names))))
plt.barh([feat_names[i] for i in order], mean_abs[order])
plt.gca().invert_yaxis()
plt.xlabel("Mean |SHAP value| (logit units)")
plt.title("Feature importance (global)")
plt.tight_layout()
plt.show()


In [0]:
import numpy as np
import matplotlib.pyplot as plt

# df = your pandas DataFrame with columns "cat__Sex_female" and target "survived"/"Survived"
x_col = "cat__Sex_female"
y_col = 'num__Survived'

x = X[x_col].astype(float).values
y = pdf[y_col].astype(float).values

# jitter so points don’t sit exactly at 0/1
rng = np.random.default_rng(42)
xj = x + rng.normal(0, 0.02, size=len(x))
yj = y + rng.normal(0, 0.02, size=len(y))

plt.figure(figsize=(6, 5))
plt.scatter(xj, yj, s=10, alpha=0.35)
plt.xticks([0, 1], ["male (0)", "female (1)"])
plt.yticks([0, 1], ["not survived (0)", "survived (1)"])
plt.xlabel(x_col)
plt.ylabel(y_col)
plt.title("cat__Sex_female vs survived (jittered scatter)")

# OPTIONAL: overlay mean survival per sex as big markers
means = pdf.groupby(x_col)[y_col].mean()
for xv, mv in means.items():
    plt.scatter([xv], [mv], s=120, marker="X")

plt.grid(True, linestyle="--", alpha=0.3)
plt.tight_layout()
plt.show()


In [0]:
import numpy as np
import matplotlib.pyplot as plt

# df = your pandas DataFrame with columns "cat__Sex_female" and target "survived"/"Survived"
x_col = "num__Pclass"
y_col = 'num__Survived'

x = X[x_col].astype(float).values
y = pdf[y_col].astype(float).values

# jitter so points don’t sit exactly at 0/1
rng = np.random.default_rng(42)
xj = x + rng.normal(0, 0.02, size=len(x))
yj = y + rng.normal(0, 0.02, size=len(y))

plt.figure(figsize=(6, 5))
plt.scatter(xj, yj, s=10, alpha=0.35)
# plt.xticks([0, 1], ["male (0)", "female (1)"])
plt.yticks([0, 1], ["not survived (0)", "survived (1)"])
plt.xlabel(x_col)
plt.ylabel(y_col)
plt.title("cat__Sex_female vs survived (jittered scatter)")

# OPTIONAL: overlay mean survival per sex as big markers
means = pdf.groupby(x_col)[y_col].mean()
for xv, mv in means.items():
    plt.scatter([xv], [mv], s=120, marker="X")

plt.grid(True, linestyle="--", alpha=0.3)
plt.tight_layout()
plt.show()

In [0]:
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from mlflow.models.signature import infer_signature
import mlflow, mlflow.sklearn, mlflow.pytorch

# --- set your UC location ---
CATALOG = "dp_ml_raw"        # or your catalog, e.g. "prod", "sandbox"
SCHEMA  = "dp_ml_titanic"     # or your schema, e.g. "ml", "models"

spark.sql(f"USE {CATALOG}.{SCHEMA}")

# --- prep ---
X_np = X.values.astype(np.float32)
y_np = pdf['num__Survived'].values.astype(int)
scaler = StandardScaler().fit(X_np)                       # fit scaler

# Logistic Regression (expects scaled input)
logreg = LogisticRegression(max_iter=1000).fit(scaler.transform(X_np), y_np)

# PyTorch model `m` already trained on scaled inputs:
#   inputs: scaler.transform(X_np).astype(np.float32)
#   outputs: probabilities in [0,1] (m ends with nn.Sigmoid)

mlflow.set_experiment("/Shared/titanic-models")

with mlflow.start_run() as run:
    # -------- 1) StandardScaler --------
    X_sig_df = X.head(50).astype(np.float64)
    sig_scaler = infer_signature(
        model_input=X_sig_df,
        model_output=scaler.transform(X_sig_df.values).astype(np.float64)
    )
    info_scaler = mlflow.sklearn.log_model(
        sk_model=scaler,
        artifact_path="scaler",
        signature=sig_scaler,
        input_example=X.head(5),
        registered_model_name='titanic-scaler'
    )

    # -------- 2) Logistic Regression (expects scaled input) --------
    Xs_sig = scaler.transform(X.head(50).values).astype(np.float64)
    sig_logreg = infer_signature(
        model_input=Xs_sig,
        model_output=logreg.predict_proba(Xs_sig)
    )
    info_logreg = mlflow.sklearn.log_model(
        sk_model=logreg,
        artifact_path="logreg",
        signature=sig_logreg,
        input_example=scaler.transform(X.head(5).values),
        registered_model_name="titanic-logreg"
    )

    # -------- 3) PyTorch NN (expects scaled input) --------
    X_t_sig = scaler.transform(X.head(50).values).astype(np.float32)
    # output example: probability column
    y_t_sig = np.zeros((X_t_sig.shape[0], 1), dtype=np.float32)
    sig_torch = infer_signature(model_input=X_t_sig, model_output=y_t_sig)

    info_torch = mlflow.pytorch.log_model(
        pytorch_model=m,
        artifact_path="torch_nn",
        signature=sig_torch,
        input_example=X_t_sig[:5],
        registered_model_name="titanic-pytorch"
    )
    mlflow.log_metric("val_accuracy_logreg", float(f"{np.mean(val_accs):.4f}"))
    mlflow.log_metric("val_accuracy_torch_nn", float(f"{np.mean(va_acc_final):.4f}"))

print("Logged URIs:")
print("  Scaler  :", info_scaler.model_uri)
print("  LogReg  :", info_logreg.model_uri)
print("  Torch NN:", info_torch.model_uri)

import mlflow
from mlflow.tracking import MlflowClient

# set alias for that new version
client = MlflowClient()



LOGREG_NAME_UC = f"{CATALOG}.{SCHEMA}.titanic-logreg"
TORCH_NAME_UC  = f"{CATALOG}.{SCHEMA}.titanic-pytorch"

if np.mean(val_accs) > np.mean(va_acc_final): 
    reg_name = LOGREG_NAME_UC
    run_id = info_logreg.run_id
else: 
    reg_name = TORCH_NAME_UC
    run_id = info_torch.run_id

# find the model version created in this run
mvs = client.search_model_versions(f"name='{reg_name}'")
this_mv = [mv for mv in mvs if mv.run_id == run_id][0]  # the version from *this* run
version = int(this_mv.version)

# set the alias
client.set_registered_model_alias(
    name=reg_name,
    alias="champion",
    version=version
)

print(f"Alias 'champion' now points to {reg_name} v{version}")

⠀⠀⠀⠀⠀⠀⢹⣄⣿⣦⣼⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⢨⣿⣿⣿⣿⠿⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⢸⣿⠟⠋⠀⠀⠘⣧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⢠⣟⣁⣴⡄⠀⠀⠀⠘⣧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⢿⠛⢉⣠⠀⠀⠀⠀⠸⣧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠈⣿⣿⠏⠀⠀⠀⠀⠀⠸⣇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠸⡇⠀⠀⠀⠀⠀⠀⠀⢹⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⣧⣤⣶⠖⠀⠀⠀⠀⠀⢻⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⢻⡉⠁⠀⣀⣤⠀⠀⠀⠀⢷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠈⣿⠛⠛⠛⠁⠀⠀⠀⠀⠘⣧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⣦⣤⣶⠿⠀⣀⣤⣤⣶⣿⣿⠒⢤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢿⠁⢀⣴⣿⣿⣿⣿⣿⣿⣿⡄⣤⣾⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⣷⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡟⣹⣿⣿⣿⣿⣿⣿⠿⣿⣿⣿⣿⣿⣿⣿⣷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣧⣿⣿⣿⣿⣿⡿⠃⠀⠀⠻⣿⣿⣿⣿⣿⣿⠛⢦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⣿⠿⠿⣿⠟⠁⠀⠀⠀⠀⠈⠻⢿⣿⣿⡇⢰⣿⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢻⡀⠀⠀⠀⠀⠀⠀⠀⢀⠀⠀⠀⠉⠉⠁⢞⣿⠈⣦⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣳⡄⠀⠀⠀⣠⣴⣾⣯⠖⠀⠀⠀⠀⠀⠹⡇⣰⣿⢧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡴⠛⠉⢻⣿⣶⣆⠈⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⢧⢉⠇⠈⢷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⢀⡴⠋⠀⢀⡴⠟⢻⡉⢿⠀⠀⣠⡤⠀⢀⡀⠀⠀⠀⠀⣾⡞⠀⠀⠀⠈⠳⣄⡀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⢀⡴⣋⣤⣀⣰⠏⠀⠀⢸⠳⣼⣦⣴⣿⣿⣿⡿⠛⠀⠀⢀⣼⠟⠀⠀⠀⠀⠀⠀⠈⠙⢦⡀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⢠⣏⣴⣿⣿⣿⣿⠀⠀⢀⣿⣶⣿⣿⣿⡋⠭⠀⠀⠀⠀⣠⠟⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⠀⠀⠀⠀
⠀⠀⠀⢀⣴⠿⠛⠛⠛⢿⣿⣿⣶⣾⣿⣿⣿⣿⣿⣿⠿⣶⣶⣴⣶⠟⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣧⠀⠀⠀
⠀⠀⠠⣿⣁⣀⡀⠀⢀⣠⣽⣿⠿⠿⠿⣿⣿⣿⣿⠏⠀⠀⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠹⡄⠀⠀
⠀⠀⠀⠈⠉⣻⠇⠀⠘⠋⠁⣀⣀⣤⣴⠾⠟⣿⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢷⠀⠀
⠀⠀⢀⣴⠟⠋⠀⢠⣶⡾⠿⠛⣛⣩⣤⣤⣤⡿⠀⠀⠀⠀⠀⠀⠀⢀⣰⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⠀⠘⡆⠀
⠀⠀⠸⣧⡀⠀⣀⠀⠀⣠⣶⠟⠛⠉⠉⠉⣽⠃⠀⠀⠀⠀⢀⣤⠾⠛⢉⡇⠀⠀⢀⣤⣶⠖⠀⠀⠀⠀⠀⠀⢸⠀⠀⢹⡀
⠀⠀⠀⠈⠛⠛⢻⡆⠀⠈⠀⣠⣶⠶⠿⢾⡟⠀⠀⠀⣠⡾⠟⠁⣠⠖⠋⣷⣠⣾⠟⠋⠁⠀⠀⠀⠀⠀⠀⠀⠈⣇⠀⠀⣇
⠀⠀⠀⠀⠀⣠⣼⠇⠀⠀⠀⠉⠀⢀⣀⣿⠃⠀⢠⠚⠉⢀⡴⠟⠁⠀⢀⣸⣿⣀⣤⣶⣶⣦⠀⠀⠀⠀⠀⠀⠀⢹⡀⠀⡿
⠀⠀⠀⠀⣾⠋⠀⠀⠀⠀⠀⠀⠐⠛⢻⡟⠀⠀⢸⣠⡶⠋⠀⠀⣠⡶⠟⠉⣿⡟⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣧⡼⠁
⠀⠀⠀⠀⠻⣦⣤⣤⣄⠀⠀⠀⠀⠀⣿⠃⠀⠀⠈⠉⠀⣀⡴⠟⠉⠀⢀⣠⣿⣷⢀⣤⣴⡶⠄⠀⠀⠀⠀⠀⠀⠀⢿⠁⠀
⠀⠀⠀⠀⠀⠀⠀⠈⣿⡀⢀⣀⣴⣾⡏⠀⠀⠀⢀⣤⣾⠋⠀⠀⣀⡴⠛⠁⠀⢻⡟⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⡇⠀
⠀⠀⠀⠀⠀⠀⠀⣀⣿⠿⠛⠉⢸⣿⠇⠀⠀⠀⠈⡏⢿⣀⣴⠞⠉⠀⠀⣀⡴⠚⣇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⡇⠀
⠀⠀⠀⢀⣤⠶⠛⠉⠀⠀⠀⠀⣼⡿⠀⠀⠀⠀⠀⠙⠼⠋⠁⠀⢀⣤⠞⠉⠀⠀⢻⣤⡾⠷⠆⠀⠀⠀⠀⠀⠀⠀⠀⣷⠀
⠀⣠⡾⠋⠁⠀⣠⣴⠿⠛⣻⡿⢿⡇⠀⠀⠀⠀⠀⠀⠀⣠⣴⡞⠋⠀⠀⠀⢀⡤⠿⣧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⠀
⣰⠏⠀⠀⠀⠘⣋⣡⠶⠛⠁⠀⢸⡇⠀⠀⠀⠀⠀⢠⢺⠋⢻⡷⣀⣀⣤⠞⠋⠀⠀⣿⣆⣀⣀⣀⡀⠀⠀⠀⠀⠀⠀⣿⠀
⠙⠷⠶⠶⠶⠛⠉⠀⠀⠀⠀⠀⢸⠇⠀⠀⠀⠀⠀⠀⢣⣣⣀⠳⡽⠋⠀⠀⠀⢀⣴⢿⣿⠛⠋⠉⠀⠀⠀⠀⠀⠀⠀⣿⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣾⠀⠀⠀⠀⠀⠀⠀⠀⠙⠓⠋⠀⠀⣀⣠⡶⠋⠀⣼⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⡏⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⢫⣿⢿⡀⠀⣠⣿⡏⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠇⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠘⠊⠙⠙⠚⠛⠛⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠀⠀